### Average all segmentations for a certain case/slice
Remove the folders that held the test models and pull the folders from wandb with the predictions. Put these in a folder /predictions/

In [1]:
import numpy as np
import os, glob
import SimpleITK as sitk
from scipy import ndimage
head = os.getcwd()


In [2]:
def get_average(case, slice):
    filename = 'case_{}_{}.npy'.format(str(case).zfill(2),str(slice).zfill(2))
    files = glob.glob(os.path.join(head, 'predictions', '*','files', 'predictions', filename))
    try:
        first_file = np.load(files[0])
        
        empty = np.zeros((*first_file.shape, 20))
        for ix, file in enumerate(files):
            prediction = np.load(file)
            print(np.max(prediction))
            # make binary
            seg = np.zeros(prediction.shape)
            seg[prediction>0.5] = 1
            # save
            empty[..., ix] = seg

        result = np.mean(empty, axis=2)
        seg = np.zeros(result.shape)
        seg[result>0.5] = 1
        return seg
    except:
        return np.zeros((256, 256))

In [14]:
# We use this function to set the original nifti information
def np2itk(arr, original_img):
    img = sitk.GetImageFromArray(arr, False)
    img.SetSpacing(original_img.GetSpacing())
    img.SetOrigin(original_img.GetOrigin())
    img.SetDirection(original_img.GetDirection())
    # this does not allow cropping (such as removing thorax, neck)
    img.CopyInformation(original_img) 
    return img

# We needed to manually rotate these IDs to get the flipping and registration working, so when done, we rotate these back to the original
angles = {}
angles[19] = -12
angles[21] = 17
angles[33] = 17
angles[5] = -18
angles[54] = 17

rotate_cases = [19, 21, 33, 5, 54]

In [15]:
def make_nifti(case):
    mtt_path = glob.glob(r'.\ISLES2018\TESTING\case_{}\SMIR.Brain.XX.O.CT_MTT.*\*.nii'.format(case))[0]
    code = mtt_path.split('.')[-2]
    mtt_nii = sitk.ReadImage(mtt_path)
    mtt_array = sitk.GetArrayFromImage(mtt_nii)
    shape = mtt_array.shape
    empty = np.zeros(shape, dtype=np.int32)
    for i in range(shape[0]):
        prediction = get_average(case, i)
        if case in rotate_cases:
            prediction = ndimage.rotate(prediction, angles[case], reshape=False,order=0)
        empty[i] = prediction
    nifti_result = np2itk(arr=empty, original_img=mtt_nii)
    result_folder = glob.glob('./results')[0]
    results_name = os.path.join(result_folder,'SMIR.case{}.{}.nii'.format(case,code))
    sitk.WriteImage(nifti_result, results_name)

In [16]:
for i in range(1,100):
    make_nifti(i)

case_01_00.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_01_01.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_00.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_01.npy
20
1.0
1.0
1.0
1.0
0.00053941563
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_02.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_03.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_04.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_05.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_06.npy
20
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_02_07.npy
20
1.0
0.9999999
1.0
1.0
0.9999999
1.0
0.9992562
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_21_00.npy
20
1.0
0.00018488061
0.9999999
0.99948406
0.9999665
0.07640698
0.00030383942
1.0
1.0
0.9961965
1.0
0.99993765
0.999982
0.0004932532
1.0
0.9889452
1.0
0.00029207763
1.0
1.0
rotate...
case_21_01.npy
20
1.0
1.0
0.0009735272
1.0
1.0
0.99988544
0.00023204168
1.0
1.0
0.774611
0.018880794
2.535585e-05
0.00022400825
1.0
1.0
0.0006951973
1.0
1.0
0.017152248
0.99957806
rotate...
case_22_00.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_22_01.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_23_00.npy
20
1.0
1.0
1.0
0.99946517
0.45085564
0.9977968
1.0
1.0
0.0068541924
1.0
1.0
0.0004864202
0.00017421726
1.0
1.0
0.00043599244
1.0
0.0018498816
1.0
0.00021823084
case_23_01.npy
20
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_24_00.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


0.0010031563
0.9997992
1.0
1.0
0.0005835084
0.019360904
0.99994826
0.99993956
1.0
0.9903503
0.99984646
0.99689156
1.0
0.00053496356
1.0
0.00031662974
1.0
0.9363666
case_48_00.npy
0
file not found..
case_48_01.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_48_02.npy
20
1.0
1.0
1.0
1.0
1.0
0.99999857
0.9999975
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
case_48_03.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99998045
case_48_04.npy
20
1.0
1.0
0.999972
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_48_05.npy
20
1.0
0.9777427
1.0
1.0
1.0
0.99346054
0.9999374
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999963
1.0
1.0
1.0
case_48_06.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
0.20396066
1.0
1.0
1.0
1.0
0.99998915
0.9999987
0.004685332
1.0
0.9991529
1.0
1.0
1.0
1.0
case_48_07.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_48_08.npy
20
1.

case_52_00.npy
0
file not found..
case_52_01.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_52_02.npy
20
1.0
1.0
1.0
1.0
0.99999905
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_52_03.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
case_52_04.npy
20
1.0
1.0
0.999992
1.0
1.0
0.9999999
0.325779
1.0
1.0
0.9999999
0.99973243
1.0
0.99993646
0.9999995
1.0
1.0
0.98442113
1.0
0.252471
0.9985933
case_52_05.npy
20
1.0
0.9987857
0.9988206
1.0
0.9994967
0.99999976
0.0030630482
1.0
1.0
0.04370131
1.0
1.0
1.0
1.0
1.0
0.9999999
0.98042
0.9947773
0.9997764
0.9999975
case_52_06.npy
20
1.0
0.9996718
0.58309066
1.0
1.0
1.0
0.9689324
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.049026262
1.0
case_52_07.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_52_08.npy
20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
case_52

1.0
1.0
1.0
case_62_10.npy
20
1.0
0.13895021
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.80745655
1.0
1.0
1.0
1.0
1.0
0.00023072309
0.0007997402
1.0
case_62_11.npy
20
1.0
4.8446345e-05
0.9941591
1.0
1.0
1.0
0.9999999
0.9999999
1.0
0.99766123
1.0
2.4542309e-05
1.0
0.41087928
1.0
0.85400677
1.0
0.00031006726
0.0034961894
1.0
case_62_12.npy
20
1.0
0.00025874397
0.011157873
0.9997632
1.0
1.0
0.000407389
1.0
1.0
0.99999714
0.99993944
3.1926167e-05
1.0
1.0
1.0
0.07030943
1.0
6.0250302e-05
0.11915202
0.23783696
case_62_13.npy
20
8.2735394e-05
1.0
0.9708584
0.9999975
1.0
0.99999356
0.0002095012
0.9966067
0.0015400816
0.00013287825
0.0047203107
9.694479e-05
0.9980248
0.99999976
1.0
0.86001176
1.0
0.00028775883
0.0022225028
0.00019421981
case_62_14.npy
20
5.2292385e-06
0.00067511154
7.10533e-06
3.0182606e-05
1.238357e-05
0.99874485
0.00052918185
3.267409e-05
0.0006918925
6.4925603e-06
0.00016963042
3.0101859e-05
0.0068222396
0.025124133
4.062075e-05
0.00039073682
0.00076702115
6.9027415e-06
6.361716e-

IndexError: list index out of range